# 05tools_4: Experiment Comparisons

Use the Python client to retrieve, review, and compare experiments from this series

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]
    - Each of these notebooks creates an experiment.  Rerunning the notebooks creates additonal runs of the experiment.  The models created by each run are uploaded to the Vertex AI Model Registry with a version alias for the experiment run name and has labels for the experiment_name and run_name.

---
## Setup

inputs:

In [9]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [10]:
REGION = 'us-central1'
SERIES = '05'

packages:

In [11]:
from google.cloud import aiplatform
import pandas as pd
from IPython.display import HTML

clients:

In [12]:
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Experiments

Get a list of all experiments in this project:

In [13]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [14]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

### Combine Runs from All Experiments in SERIES

Combine the runs from all experiments in SERIES into a single dataframe:

In [15]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05i-tf-classification-dnn
experiment-05-05h-tf-classification-dnn
experiment-05-05g-tf-classification-dnn
experiment-05-05f-tf-classification-dnn
experiment-05-05e-tf-classification-dnn
experiment-05-05d-tf-classification-dnn
experiment-05-05c-tf-classification-dnn
experiment-05-05b-tf-classification-dnn
experiment-05-05a-tf-classification-dnn
experiment-05-05-tf-classification-dnn


In [16]:
results

,experiment_name,run_name,run_type,state,param.var_target,param.hyperparameter.momentum,param.model.save,param.training.prefetch,param.hyperparameter.learning_rate,param.hyperparameterTuning.link,...,time_series_metric.val_accuracy,time_series_metric.val_auprc,time_series_metric.train_loss,param.customJobs.link,param.customJobs.tensorboard,param.training.epochs,param.trainingPipelines.display_name,param.trainingPipelines.resource_name,param.customJobs.display_name,param.customJobs.resource_name
0,experiment-05-05i-tf-classification-dnn,run-20220826194138-18,system.ExperimentRun,COMPLETE,Class,2.356476e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.001000,https://console.cloud.google.com/ai/platform/l...,...,0.999079,0.999399,0.005853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-tf-classification-dnn,run-20220826194138-17,system.ExperimentRun,COMPLETE,Class,4.584998e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.033156,https://console.cloud.google.com/ai/platform/l...,...,0.999292,0.999717,0.002540,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-tf-classification-dnn,run-20220826194138-16,system.ExperimentRun,COMPLETE,Class,8.117497e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.008662,https://console.cloud.google.com/ai/platform/l...,...,0.999292,0.999670,0.002691,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-tf-classification-dnn,run-20220826194138-15,system.ExperimentRun,COMPLETE,Class,7.285184e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.001000,https://console.cloud.google.com/ai/platform/l...,...,0.999186,0.999432,0.004371,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-tf-classification-dnn,run-20220826194138-14,system.ExperimentRun,COMPLETE,Class,6.349445e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.003615,https://console.cloud.google.com/ai/platform/l...,...,0.999256,0.999576,0.003417,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,experiment-05-05i-tf-classification-dnn,run-20220826194138-13,system.ExperimentRun,COMPLETE,Class,2.256144e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.027179,https://console.cloud.google.com/ai/platform/l...,...,0.999292,0.999671,0.002745,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,experiment-05-05i-tf-classification-dnn,run-20220826194138-12,system.ExperimentRun,COMPLETE,Class,1.000000e-07,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.001000,https://console.cloud.google.com/ai/platform/l...,...,0.998938,0.999134,0.008103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,experiment-05-05i-tf-classification-dnn,run-20220826194138-11,system.ExperimentRun,COMPLETE,Class,1.000000e-07,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.073547,https://console.cloud.google.com/ai/platform/l...,...,0.999327,0.999622,0.002491,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,experiment-05-05i-tf-classification-dnn,run-20220826194138-10,system.ExperimentRun,COMPLETE,Class,9.000000e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.002678,https://console.cloud.google.com/ai/platform/l...,...,0.999292,0.999531,0.002907,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,experiment-05-05i-tf-classification-dnn,run-20220826194138-9,system.ExperimentRun,COMPLETE,Class,9.000000e-01,gs://statmike-mlops-349915/fraud/models/05/05i...,1.0,0.028017,https://console.cloud.google.com/ai/platform/l...,...,0.999292,0.999622,0.002272,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get Links to all Experiment Runs with Vertex AI Training Jobs:
- Link to Job
- Link to TensorBoard for Job

In [18]:
# just names and training job columns
mask = results.columns.str.startswith(('param.trainingPipeline','param.customJobs','param.hyperparameterTuningJobs'))
mask[0:2] = True
HTML(results.loc[:,mask].to_html(render_links=True, escape=False))

,experiment_name,run_name,param.hyperparameterTuningJobs.display_name,param.customJobs.link,param.customJobs.tensorboard,param.trainingPipelines.display_name,param.trainingPipelines.resource_name,param.customJobs.display_name,param.customJobs.resource_name
0,experiment-05-05i-tf-classification-dnn,run-20220826194138-18,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-tf-classification-dnn,run-20220826194138-17,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-tf-classification-dnn,run-20220826194138-16,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-tf-classification-dnn,run-20220826194138-15,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-tf-classification-dnn,run-20220826194138-14,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
5,experiment-05-05i-tf-classification-dnn,run-20220826194138-13,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
6,experiment-05-05i-tf-classification-dnn,run-20220826194138-12,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
7,experiment-05-05i-tf-classification-dnn,run-20220826194138-11,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
8,experiment-05-05i-tf-classification-dnn,run-20220826194138-10,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN
9,experiment-05-05i-tf-classification-dnn,run-20220826194138-9,05i_fraud_20220826194138,NaN,NaN,NaN,NaN,NaN,NaN


### Hyperlinks To TensorBoard for Experiments and Comparison Of All Experiments

Get a hyperlink to the TensorBoard for each experiment:

In [19]:
tensorboards = []
for experiment in experiments:
    tboard = experiment.get_backing_tensorboard_resource().resource_name.replace('/', '+')
    tensorboards.append([experiment.name, f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tboard}+experiments+{experiment.name}"])
tensorboards = pd.DataFrame(tensorboards, columns = ['Experiment', 'TensorBoard Link'])    

In [20]:
HTML(tensorboards.to_html(render_links=True, escape=False))

,Experiment,TensorBoard Link
0,experiment-05-05i-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05i-tf-classification-dnn
1,experiment-05-05h-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05h-tf-classification-dnn
2,experiment-05-05g-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05g-tf-classification-dnn
3,experiment-05-05f-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05f-tf-classification-dnn
4,experiment-05-05e-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05e-tf-classification-dnn
5,experiment-05-05d-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05d-tf-classification-dnn
6,experiment-05-05c-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05c-tf-classification-dnn
7,experiment-05-05b-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05b-tf-classification-dnn
8,experiment-05-05a-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05a-tf-classification-dnn
9,experiment-05-05-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05-tf-classification-dnn


Compare all experiments in SERIES with TensorBoard:

In [21]:
compare_link = f"https://{REGION}.tensorboard.googleusercontent.com/compare/"
for e, experiment in enumerate(experiments):
    if e>0: compare_link += ','
    tboard = experiment.get_backing_tensorboard_resource().resource_name.split('/')
    compare_link += f"{e+1}-{experiment.name}:{'+'.join(tboard[1::2])}+{experiment.name}"
print(compare_link)

https://us-central1.tensorboard.googleusercontent.com/compare/1-experiment-05-05i-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05i-tf-classification-dnn,2-experiment-05-05h-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05h-tf-classification-dnn,3-experiment-05-05g-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05g-tf-classification-dnn,4-experiment-05-05f-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05f-tf-classification-dnn,5-experiment-05-05e-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05e-tf-classification-dnn,6-experiment-05-05d-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05d-tf-classification-dnn,7-experiment-05-05c-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05c-tf-classification-dnn,8-experiment-05-05b-tf-classification-dnn:

## Rank Experiment Runs on a Metric

In [22]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', metric]].copy().reset_index(drop = True)
    ranks['rank'] = ranks[metric].rank(method='dense', ascending = False)
    return ranks.sort_values(by = ['rank'])
    
ranks = ranker()

In [23]:
ranks

,experiment_name,run_name,metric.test_auprc,rank
17,experiment-05-05i-tf-classification-dnn,run-20220826194138-7,0.999812,1.0
43,experiment-05-05g-tf-classification-dnn,run-20220826184958-4,0.999766,2.0
37,experiment-05-05h-tf-classification-dnn,run-20220826194057-3,0.999721,3.0
5,experiment-05-05i-tf-classification-dnn,run-20220826194138-13,0.999721,4.0
19,experiment-05-05h-tf-classification-dnn,run-20220826194057-19,0.999721,5.0
40,experiment-05-05g-tf-classification-dnn,run-20220826184958-8,0.999720,6.0
18,experiment-05-05h-tf-classification-dnn,run-20220826194057-20,0.999720,7.0
13,experiment-05-05i-tf-classification-dnn,run-20220826194138-4,0.999674,8.0
1,experiment-05-05i-tf-classification-dnn,run-20220826194138-17,0.999674,9.0
38,experiment-05-05g-tf-classification-dnn,run-20220826184958-10,0.999674,9.0
